In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from PIL import Image
from IPython.core.display import HTML
from functools import partial

In [2]:
%reload_ext autoreload
%autoreload 2
from engine.utils_nav import ProgramGenerator, ProgramInterpreter
from prompts.gqa import create_prompt

/home/ziliottf/anaconda3/envs/visprog/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

from engine.step_interpreters_nav import parse_step

eval_ = """EVAL0=EVAL(expr="'STOP' if {LOC0} > 0 else 'NAVIGATE'")"""
stop = """STOP0=STOP(var=EVAL0)"""
# str_2 = """ANSWER2=EVAL(expr="'yes' if {ANSWER0} + {ANSWER1} > 0 else 'no'")"""
dict_ = parse_step(eval_, partial=False)
dict_

In [120]:
question = """ go to [0,5,3]"""
answer= """while not STOP0:
    NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')
    LOC0=LOCATE(pos=NAV0)
    EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
    STOP0=STOP(var=EVAL0)
while not STOP1:
    NAV1=NAVIGATE(image=IMAGE,goal='[0,0,3]')
    LOC1=LOCATE(pos=NAV1)
    EVAL1=EVAL(expr="'STOP' if {LOC1} else 'NAVIGATE'")
    STOP1=STOP(var=EVAL1)
"""

image = Image.open('../assets/startnav.png')
image.thumbnail((640,640),Image.Resampling.LANCZOS)
init_state = dict(
    IMAGE=image.convert('RGB')
)

prompter = partial(create_prompt,method='all')
generator = ProgramGenerator(prompter=prompter, debug=True)

interpreter = ProgramInterpreter(dataset='pointnav')
prog,_ = generator.generate(dict(question=question), answer) #fake answer
print(prog)

Registering NAVIGATE step
Registering STOP step
Registering LOCATE step
Registering EVAL step
while not STOP0:
    NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')
    LOC0=LOCATE(pos=NAV0)
    EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
    STOP0=STOP(var=EVAL0)
while not STOP1:
    NAV1=NAVIGATE(image=IMAGE,goal='[0,0,3]')
    LOC1=LOCATE(pos=NAV1)
    EVAL1=EVAL(expr="'STOP' if {LOC1} else 'NAVIGATE'")
    STOP1=STOP(var=EVAL1)


In [123]:
result, prog_state = interpreter.execute(prog,\
                                        init_state,\
                                        inspect=False)
result

-------------
| Trial n°1 |
-------------
NAVIGATE
0.03237647012238265
LOCATE
EVAL
STOP
{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512 at 0x7F355C673100>, 'NAV0': 0.03237647012238265, 'LOC0': True, 'EVAL0': 'STOP', 'STOP0': True}
-------------
| Trial n°1 |
-------------
NAVIGATE
0.8927929461214491
LOCATE
EVAL
STOP
-------------
| Trial n°2 |
-------------
NAVIGATE
0.2437314293170343
LOCATE
EVAL
STOP
{'IMAGE': <PIL.Image.Image image mode=RGB size=512x512 at 0x7F355C673100>, 'NAV0': 0.03237647012238265, 'LOC0': True, 'EVAL0': 'STOP', 'STOP0': True, 'NAV1': 0.2437314293170343, 'LOC1': True, 'EVAL1': 'STOP', 'STOP1': True}


True

In [118]:
answer= """while not STOP0:
    NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')
    LOC0=LOCATE(pos=NAV0)
    EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
    STOP0=STOP(var=EVAL0)
while not STOP1:
    NAV1=NAVIGATE(image=IMAGE,goal='[0,0,3]')
    LOC1=LOCATE(pos=NAV1)
    EVAL1=EVAL(expr="'STOP' if {LOC1} else 'NAVIGATE'")
    STOP1=STOP(var=EVAL1)
"""

In [119]:
import re
def clear_splits(prog_str):
    # Use regular expressions to find and extract the instructions with initial whitespace removed
    instructions = re.findall(r'while not \w+:\n\s*(.*?)(?=\nwhile not|\Z)', prog_str, re.DOTALL)

    # Create a dictionary to organize the instructions
    instructions_dict = {}

    for split_counter, instr in enumerate(instructions, start=1):
        cleaned_instruction = instr.rstrip('\n').replace('\\', '')
        cleaned_lines = [line.strip() for line in cleaned_instruction.split('\n')]
        instructions_dict[f'split{split_counter}'] = cleaned_lines

    return instructions_dict

clear_splits(answer)

{'split1': ["NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')",
  'LOC0=LOCATE(pos=NAV0)',
  'EVAL0=EVAL(expr="\'STOP\' if {LOC0} else \'NAVIGATE\'")',
  'STOP0=STOP(var=EVAL0)'],
 'split2': ["NAV1=NAVIGATE(image=IMAGE,goal='[0,0,3]')",
  'LOC1=LOCATE(pos=NAV1)',
  'EVAL1=EVAL(expr="\'STOP\' if {LOC1} else \'NAVIGATE\'")',
  'STOP1=STOP(var=EVAL1)']}

In [89]:
# Your input string
input_str = """while not STOP0:
    NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')
    LOC0=LOCATE(pos=NAV0)
    EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
    STOP0=STOP(var=EVAL0)
while not STOP1:
    NAV1=NAVIGATE(image=IMAGE,goal='[0,0,3]')
    LOC1=LOCATE(pos=NAV1)
    EVAL1=EVAL(expr="'STOP' if {LOC1} else 'NAVIGATE'")
    STOP1=STOP(var=EVAL1)
"""

# Use regular expressions to find and extract the instructions with initial whitespace removed
instructions = re.findall(r'while not \w+:\n\s*(.*?)(?=\nwhile not|\Z)', input_str, re.DOTALL)

# Create a dictionary to organize the instructions
instructions_dict = {}

# Initialize a counter for the keys
split_counter = 1

# Remove backslashes and split the instructions, while removing leading whitespace
for instr in instructions:
    cleaned_instruction = instr.rstrip('\n').replace('\\', '')
    key = f'split{split_counter}'
    lines = cleaned_instruction.split('\n')
    cleaned_lines = [line.strip() for line in lines]
    instructions_dict[key] = cleaned_lines
    split_counter += 1

# Print the instructions organized in a dictionary without leading whitespace
for key, value in instructions_dict.items():
    print(f"{key}:")
    for instr in value:
        print(f"    {instr}")






split1:
    NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')
    LOC0=LOCATE(pos=NAV0)
    EVAL0=EVAL(expr="'STOP' if {LOC0} else 'NAVIGATE'")
    STOP0=STOP(var=EVAL0)
split2:
    NAV1=NAVIGATE(image=IMAGE,goal='[0,0,3]')
    LOC1=LOCATE(pos=NAV1)
    EVAL1=EVAL(expr="'STOP' if {LOC1} else 'NAVIGATE'")
    STOP1=STOP(var=EVAL1)


In [90]:
instructions_dict['split1']

["NAV0=NAVIGATE(image=IMAGE,goal='[5,0,0]')",
 'LOC0=LOCATE(pos=NAV0)',
 'EVAL0=EVAL(expr="\'STOP\' if {LOC0} else \'NAVIGATE\'")',
 'STOP0=STOP(var=EVAL0)']